In [1]:
import os

In [2]:
%pwd

'c:\\Users\\ROOT\\Desktop\\mlops_projects\\data_science_project\\research'

In [3]:
os.chdir("../")

In [4]:
%ls

 Volume in drive C has no label.
 Volume Serial Number is 3C50-BC77

 Directory of c:\Users\ROOT\Desktop\mlops_projects\data_science_project

05/20/2025  09:21 AM    <DIR>          .
05/20/2025  09:21 AM    <DIR>          ..
05/20/2025  06:33 AM    <DIR>          .github
05/20/2025  09:00 AM                24 .gitignore
05/21/2025  01:03 AM    <DIR>          artifacts
05/20/2025  06:33 AM    <DIR>          config
05/20/2025  06:36 AM                 0 DockerFile
05/20/2025  06:02 AM            35,823 LICENSE
05/20/2025  08:25 AM    <DIR>          logs
05/21/2025  12:55 AM             1,435 main.py
05/20/2025  08:56 AM                11 params.yaml
05/21/2025  12:17 AM               399 README.md
05/20/2025  06:11 AM               132 requirements.txt
05/21/2025  01:05 AM    <DIR>          research
05/20/2025  04:19 PM               333 schema.yaml
05/20/2025  06:36 AM                 0 setup.py
05/20/2025  06:33 AM    <DIR>          src
05/20/2025  09:07 AM             1,452 template.p

'DOSKEY' is not recognized as an internal or external command,
operable program or batch file.


In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    alpha: float
    l1_ratio: float
    target_column: str

In [8]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml, create_directories

In [12]:
class ConfigurationManager:
    def __init__(self,
                config_file_path=CONFIG_FILE_PATH,
                params_file_path=PARAMS_FILE_PATH,
                schema_file_path=SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        self.schema = read_yaml(schema_file_path)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])
            
        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path=config.train_data_path,
            test_data_path=config.test_data_path,
            model_name=config.model_name,
            alpha=params.alpha,
            l1_ratio=params.l1_ratio,
            target_column=schema.name
            )
        return model_trainer_config


In [13]:
import pandas as pd 
import os
from src.datascience import logger
from sklearn.linear_model import ElasticNet
import joblib

In [14]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        train_X = train_data.drop(columns=[self.config.target_column], axis=1)
        test_X = test_data.drop(columns=[self.config.target_column],axis=1)
        train_y = train_data[[self.config.target_column]]
        test_y = test_data[[self.config.target_column]]

        lr = ElasticNet(
            alpha=self.config.alpha,
            l1_ratio=self.config.l1_ratio,
            random_state=42
        )
        lr.fit(train_X,train_y)

        joblib.dump(lr, os.path.join(self.config.root_dir,self.config.model_name))


In [15]:
try:
    config = ConfigurationManager()
    model_train_config = config.get_model_trainer_config()
    model_train = ModelTrainer(config=model_train_config)
    model_train.train()
except Exception as e:
    raise e

[2025-05-21 01:31:06,213: INFO: common: Yaml File: config\config.yaml loaded successfully]
[2025-05-21 01:31:06,216: INFO: common: Yaml File: params.yaml loaded successfully]
[2025-05-21 01:31:06,221: INFO: common: Yaml File: schema.yaml loaded successfully]
[2025-05-21 01:31:06,223: INFO: common: Created Directory at: artifacts]
[2025-05-21 01:31:06,225: INFO: common: Created Directory at: artifacts/model_trainer]
